In [1]:
from deck import *
from game import *
from player import *
from phases import Phases
import random

In [2]:
data = {
    "Silver": [],
    "Gold": []
}
# Tuple State, Legal_actions, action, reward

In [3]:

policies = [None,None]
wins=[0,0]
n_games = 100


def get_move(index, state, move_indexes=None, move_strings=None):
    assert(move_indexes != None or move_strings != None), "at least one list must not be None {} {}".format(move_indexes,move_strings)

    if (move_indexes != None):
        length = len(move_indexes)
    else:
        length = len(move_strings)

    if policies[index] == None or length == 1:
        if move_indexes != None:
            move = random.sample(move_indexes,1)[0]
        else:
            move = random.sample(move_strings,1)[0]
    else:
        assert (False), "TODO"
        pass
    return move

In [33]:
# Simulate n_names times
for i in range(5000):
    # Create game
    #game = Game([Player(get_8ed_core_gold_deck(), name="Gold"), Player(get_8ed_core_silver_deck(), name="Silver")])
    game = Game([Player(get_8ed_core_gold_deck(), name="Player1", deck_name="Gold"), Player(get_8ed_core_silver_deck(), name="Player2", deck_name="Silver")])
    game.start_game()

    game_data = {
        game.players[0].name: [],
        game.players[1].name: []
    }
    while not game.is_over():
        # Get player, valid moves
        player = game.player_with_priority
        move_indexes, move_strings = game.get_moves()
        state = game.get_board_string()

        # Skip COMBAT_DAMAGE_STEP_510_1c, no choices in damage assignment
        if game.current_phase_index == Phases.COMBAT_DAMAGE_STEP_510_1c:
            assert(len(move_indexes) == 1), "Should only be 1 value move"
            game.make_move(move_indexes[0],False)
            continue

        # Get move, may have to unroll action
        if isinstance(move_strings, ActionUnroller):
            unroller = move_strings
            while not unroller.is_done():
                moves = unroller.get_legal_moves()
                move_string = get_move(player.index,state, move_indexes=None, move_strings=moves)

                # Record Move
                if len(moves) > 1:
                    game_data[player.name].append([state, moves, move_string])

                #print(move_string)
                state = unroller.register_move(move_string)
            # Apply unrolled actions to game
            unroller.make_move()
        else:
            #print(state)
            move = get_move(player.index, state, move_indexes, move_strings)
            move_string = move_strings[move_indexes.index(move)]

            # Record Move
            if len(move_indexes) > 1:
                game_data[player.name].append([state, move_strings, move_string])

            #print(move_string)
            game.make_move(move, False)
        #move = mcts.uct(game, itermax=5)

    if game.players[1].has_lost:
        wins[0] += 1
        reward_0 = 1
        reward_1 = -1
        #print("State", state)
    elif game.players[0].has_lost:
        wins[1] += 1
        reward_0 = -1
        reward_1 = 1
        #print("State", state)
    # Add reward info
    name_0 = game.players[0].name
    name_1 = game.players[1].name
    len_0 = len(game_data[name_0])
    len_1 = len(game_data[name_1])
    for i in range(max(len_0, len_1)):
        if i <= len_0 - 1:
            game_data[name_0][i].append(reward_0)
        if i <= len_1 - 1:
            game_data[name_1][i].append(reward_1)
    # Add game_data to overall data
    for player in game.players:
        data[player.deck_name].append(game_data[player.name])
    
print("player 0 wins: %i, player 1 wins: %i" % (wins[0], wins[1]))


player 0 wins: 1896, player 1 wins: 3904


In [5]:
#np.array(game_data['Player1'])

<ipython-input-5-2a37014f8a44>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array(game_data['Player1'])


array([['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Lava Axe$Stone Rain$Volcanic Hammer$Goblin Raider$Forest$Mountain$Hill Giant$Hill Giant$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Island$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
        list(['Forest', 'Mountain', 'Pass']), 'Pass', 1],
       ['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Lava Axe$Stone Rain$Volcanic Hammer$Goblin Raider$Forest$Mounta

In [12]:
len(data['Silver'])

100

In [19]:
data['Silver'][0][0]

['\n        player-color$ Silver$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Island$Island$Island$Giant Octopus$Vizzerdrix$Plains$Plains$Island$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 6$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ Mountain$\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['Island', 'Island', 'Island', 'Plains', 'Plains', 'Island', 'Pass'],
 'Island',
 -1]

In [22]:
data['Gold'][0][0]

['\n        player-color$ Gold$\n        life$ 20$\n        opponent-life$ 20$\n        phase$ Phases.MAIN_PHASE_PRE_COMBAT$\n        hand$ Ogre Taskmaster$Hill Giant$Volcanic Hammer$Norwood Ranger$Grizzly Bears$Mountain$Mountain$\n        generic-debt$ 0$\n        manapool$ White$ 0$ Blue$ 0$ Black$ 0$ Red$ 0$ Green$ 0$ Colorless$ 0$\n        opponent-cards$ 7$\n        self-lands-untapped$ $\n        self-lands-tapped$ $\n        opponent-lands-untapped$ $\n        opponent-lands-tapped$ $\n        self-creatures-untapped$ $\n        self-creatures-tapped$ $\n        opponent-creatures-untapped$ $\n        opponent-creatures-tapped$ $\n        attackers-blockers$ None$\n        damage-order$ None$\n        ',
 ['Mountain', 'Mountain', 'Pass'],
 'Mountain',
 1]

In [32]:
wins

[268, 532]

In [37]:
# Count total amount of transitions
total_transitions = 0
for deck_name, deck_data in data.items():
    for game_data in deck_data:
        total_transitions += len(game_data)
print(total_transitions)

901408


In [40]:
flat_data = []
for deck_type, deck_data in data.items():
    for game in deck_data:
        flat_data.extend(game)

In [42]:
len(flat_data)

901408

In [58]:
flat_array = np.array(flat_data, dtype=object)

In [63]:
np.savez_compressed('./data/test_data', flat_array)

In [55]:
actions = set(flat_array[:, 2])

In [57]:
print(actions)

{'Sacred Nectar', 'Eager Cadet$0$block$Spined Wurm$0', 'Norwood Ranger', 'Enormous Baloth$1$block$Giant Octopus$1', 'Coral Eel$1$block$Enormous Baloth$1', 'Fugitive Wizard$0$block$Hill Giant$1', 'tapped_ability_0_Mountain', 'Norwood Ranger$1$block$Glory Seeker$0', 'Glory Seeker$3$block$Goblin Raider$1', 'Hill Giant$0$block$Fugitive Wizard$0', 'Coral Eel$0$block$Hill Giant$1', 'Fugitive Wizard$0$block$Grizzly Bears$0', 'Vizzerdrix$1$block$Hill Giant$1', 'Eager Cadet$0$block$Enormous Baloth$1', 'Giant Octopus$1$block$Norwood Ranger$1', 'Glory Seeker$0', 'Norwood Ranger$0$block$Giant Octopus$2', 'Eager Cadet$0$block$Grizzly Bears$0', 'tapped_ability_0_Island', 'Spined Wurm$0$block$Fugitive Wizard$0', 'Glory Seeker$0$block$Norwood Ranger$0', 'Giant Octopus$0$block$Grizzly Bears$1', 'Glory Seeker$2$block$Grizzly Bears$0', 'Glory Seeker$2$block$Enormous Baloth$1', 'Vizzerdrix$1$block$Enormous Baloth$1', 'Norwood Ranger$0$block$Coral Eel$1', 'Glory Seeker$1$block$Enormous Baloth$0', 'Fugitive